In [1]:
##get data
import numpy as np
import sys
sys.path
sys.path.append('../models')
import pickle
import math
import tensorflow as tf
import gpflow
import random

In [2]:
##set grid
u1 = np.arange(.1,1.1,.1)
u2 = np.arange(.1,1.1,.1)
rho = np.arange(.8,1.1,.05)

In [3]:
v1 = np.arange(1,10,1)
v2 = np.arange(10,10,1)

In [4]:

noise = np.arange(400.0,500.0,100.0)


In [5]:
with open('../../Downloads/processed_data_for_gridsearch_GP_onehot_random.pkl','rb') as f:
    data = pickle.load(f)
    

In [6]:
X=data[0]
y=data[1]

In [7]:
def rbf_custom_np( X, X2=None):
        #print(X)
        #print(X2)
        if X2 is None:
             X2=X
        return math.exp(-((X-X2)**2)/1.0)
        #tf.divide(tf.square(-tf.subtract(X,X2)),tf.constant(1.0,dtype=tf.float64)))


In [8]:
import CustomKernelStatic


In [51]:
X[900][-3:]

[-0.0, 4.0, 37.0]

In [9]:
def get_likelihood(u1,u2,s1,s2,rho,noise,X,y,rhos,users):
    sess = tf.Session()
    k =  CustomKernelStatic.CustomKernelStatic(194,mysession=sess,rhos=rhos,select_users=users,baseline_indices=[i for i in range(192)],\
                  psi_indices=[0,64],user_day_index=193,user_index=192,num_data_points=len(X),initial_u1=u1,initial_u2=u2,\
                           initial_s1=s1,initial_s2=s2,initial_rho=rho,initial_noise=noise)
  
    m = gpflow.models.GPR(X,y, kern=k)
    m.initialize(session=sess)
 
    try:
        l =m.compute_log_likelihood()
        print('huzzah')
        print(l)
    except:
        l=-100
        print('not huzzah not')
        print(l)
    sess.close()
    return l
    ##get likelihood #return likelihood

In [10]:
def run():
    to_save = {}
    u1 = np.arange(.1,1.1,.1)
    u2 = np.arange(.1,1.1,.1)
    first_rhos = np.arange(.8,1.1,.05)
    v1 = np.arange(1,10,1)
    v2 = np.arange(1,10,1)
    
    noise = np.arange(400.0,600.0,100.0)
    
    with open('../../Downloads/processed_data_for_gridsearch_GP_onehot.pkl','rb') as f:
        data = pickle.load(f)
    
    X=data[0]
    y=data[1]
    #for x in X:
        #x[0]=1+random.random()/.01
   # for x in X:
       # x[6]=1+random.random()/.01
    newX = []
    for x in X:
        newX.append(np.array([float(i) for i in x]))
    newX=np.array(newX)
    
    rdayone = [x[193] for x in newX]
    rdaytwo = rdayone
    rhos = np.array([[rbf_custom_np( rdayone[i], X2=rdaytwo[j]) for j in range(len(X))] for i in range(len(X))])
    users = np.array([[float(newX[i][192]==newX[j][192]) for j in range(len(X))] for i in range(len(X))])
    
    print(X.shape)
    print(y.shape)
    print(len(set(X[:,0])))
    for uone in u1:
        for utwo in u2:
            for rho in first_rhos:
                for s1 in v1:
                    for s2 in v2:
                        for n in noise:
                            likli=get_likelihood(uone,uone,s1,s2,rho,n,newX,y,rhos,users)
                             
                            key = '-'.join([str(i) for i in[uone,utwo,rho,s1,s2,n]])
                            to_save[key]=likli
                            
    return to_save

In [13]:
 #run()
    

In [13]:
    X=data[0]
    y=data[1]
    for x in X:
        x[0]=1+random.random()
    for x in X:
        x[6]=1+random.random()

In [11]:
newX = []
for x in X:
    newX.append(np.array([float(i) for i in x]))
newX=np.array(newX)
#newX=newX.astype(float)


In [12]:
    rdayone = [x[193] for x in newX]
    rdaytwo = rdayone
    rhos = np.array([[rbf_custom_np( rdayone[i], X2=rdaytwo[j]) for j in range(len(X))] for i in range(len(X))])
    users = np.array([[float(newX[i][192]==newX[j][192]) for j in range(len(X))] for i in range(len(X))])
    

In [85]:
set([type(y[i][0]) for i in range(len(y))])

{numpy.float64}

In [86]:
set([type(i) for i in newX[0]])

{numpy.float64}

In [93]:
type(noise[0])

numpy.int64

In [13]:
    sess = tf.Session()
    k =  CustomKernelStatic.CustomKernelStatic(194,mysession=sess,rhos=rhos,select_users=users,baseline_indices=[i for i in range(192)],\
                  psi_indices=[0,64],user_day_index=193,user_index=192,num_data_points=len(X),initial_u1=u1[0],initial_u2=u2[0],\
                           initial_s1=v1[0],initial_s2=20,initial_rho=rho[0],initial_noise=noise[0])
  
    m = gpflow.models.GPR(newX,y, kern=k)
    m.initialize(session=sess)

In [14]:
gpflow.train.ScipyOptimizer().minimize(m,session=sess)
lik=m.compute_log_likelihood()



In [15]:
m.kern.sigma_u1.value

array(24492.49426332)

In [16]:
m.kern.sigma_u2.value

array(2231.29672266)

In [17]:
m.kern.sigma_v.value

array([[[5.21906177e+02, 0.00000000e+00],
        [0.00000000e+00, 1.00000000e-06]]])

In [18]:
m.kern.sigma_rho.value

array(2.)

In [19]:
m.kern.noise_term.value

array(95224.65812823)

In [22]:
get_sigma_u(24492.49426332,2231.29672266,2)

array([[24492.49426332,  7392.56533143],
       [ 7392.56533143,  2231.29672266]])

In [21]:
def get_sigma_u(u1,u2,rho):
    off_diagaonal_term = u1**.5*u2**.5*(rho-1)
    return np.array([[u1,off_diagaonal_term],[off_diagaonal_term,u2]])

In [15]:
sess.close()

NameError: name 'sess' is not defined

In [22]:
lik

-1263332.3926229083

In [98]:
lik

-2450425.7153464314

In [23]:
lik


-1269271.1689228236

In [49]:
for x in X:
    x[0]=1



In [103]:
np.array([yy[0] for yy in y]).std()

407.30819143947593

In [91]:
for i in range(len(X[0])):
    print(i)
    print(len(set(X[:,i])))
    print('\n')

0
1


1
2


2
2


3
4


4
2


5
2


6
1


7
2


8
2


9
4


10
2


11
2


12
2


13
3


14
3


15
7


16
3


17
3


18
36


19
62




In [24]:
    m = np.eye(len(None))
    m = np.add(m,.1)

TypeError: object of type 'NoneType' has no len()